In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from tqdm import tqdm
import scipy

"""
The format of the getAccel() and main() functions are largely inspired by the code written by 
Philip Mocz (2020) Princeton Univeristy, @PMocz in his Create Your Own Plasma PIC Simulation (With Python)
https://github.com/pmocz/pic-python/blob/master/pic.py
The phi_solve() function is taken from a Mathcube article and given in 
https://gist.github.com/mathcube7/fc33438a8b10af0100bcde9858d1f239#file-poisson-1-py


This cell contains the function which solves the poisson equation at each time step and the function that calculates
the accleration of each particle at each time step. Both the FFT (phi_compute) and finite differencing (phi_solve) 
method functions are present, although only the phi_compute() function is used in this notebook. 

"""

#function which calculates electric potential from charge density using FFT
def phi_compute(rho,eps0,Nx,Ny,dx,dy):
    kx = 2*np.pi*scipy.fft.fftfreq(Nx, d=dx)
    ky = 2*np.pi*scipy.fft.fftfreq(Ny, d=dy)
    
    #calculate fft of charge density
    rhok = scipy.fft.fft2(rho, workers = 8)
    phik = np.zeros((Nx,Ny), dtype = np.complex_)
    
    #calculate fourier transform of electric potential
    for i in range(Nx):
        for j in range(Ny):
            if kx[i] == 0 and ky[j] == 0:
                phik[i,j] = 0
            else:
                phik[i,j] = -rhok[i,j]/(kx[i]**2 + ky[j]**2)
                
    phik = 1 / eps0 * phik 
    phi = np.real(scipy.fft.ifft2(phik, workers = 8)) 

    return phi

#function which interatively calculates electric potential from charge density using finite differencing
def phi_solve(f, rho, h, stepper, atol=1.E-6, max_steps=10**5):
    for _ in range(max_steps):
        f_new = stepper(f, rho, h)
        if np.max(np.abs(f_new - f)) < atol:
            return f_new
        f = f_new
    return f

#function which performs one finite difference step
def roll_step(f, rho, h):
    f_new = 0.25 * (np.roll(f, 1, axis = 0) + np.roll(f, -1, axis = 0) + np.roll(f, 1, axis = 1) + np.roll(f, -1, axis = 1)) - 0.25 * h**2 * rho
    return f_new
    
#function which calculates acceleration of each particle 
def getAccel(pos,Nx,Ny,Lx,Ly,dt,n0,N):
    eps0 = 1
    dx = Lx/Nx
    dy = Ly/Ny
    j = np.zeros((N,2))
    jp1 = np.zeros((N,2))
    
    #define grid points
    j[:,0] = np.floor(pos[:,0]/dx).astype(int)
    j[:,1] = np.floor(pos[:,1]/dy).astype(int)
    jp1 = j + 1
    
    #calculate weights of particle on each grid point
    weight_jxjy = (jp1[:,0]*dx - pos[:,0])/dx * (jp1[:,1]*dy - pos[:,1])/dy
    weight_jxp1jy = (pos[:,0] - j[:,0]*dx)/dx * (jp1[:,1]*dy - pos[:,1])/dy
    weight_jxjyp1 = (jp1[:,0]*dx - pos[:,0])/dx * (pos[:,1] - j[:,1]*dy)/dy
    weight_jxp1jyp1 = (pos[:,0] - j[:,0]*dx)/dx * (pos[:,1] - j[:,1]*dy)/dy
    
    #periodic boundary conditions:
    jp1[:,0] = np.mod(jp1[:,0], Nx)
    jp1[:,1] = np.mod(jp1[:,1], Ny)
    
    #smear the particle position across four nearest gridpoints, weighting by distance to each gridpoint 
    xbin = np.linspace(0, Nx, Nx+1)
    ybin = np.linspace(0, Ny, Ny+1)
    
    #count number of particles smeared to each grid point
    n = st.binned_statistic_2d(j[:,0], j[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j,j) grid point
    n += st.binned_statistic_2d(jp1[:,0], jp1[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j+1,j+1) grid point
    n += st.binned_statistic_2d(j[:,0], jp1[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j,j+1) grid point
    n += st.binned_statistic_2d(jp1[:,0], j[:,1], None, 'count', bins = [xbin,ybin], expand_binnumbers=True).statistic #(j+1,j) grid point
    
    #weight the contribution  of each particle to each grid point it has been smeared across
    n[j[:,0].astype(int),j[:,1].astype(int)] *= weight_jxjy
    n[jp1[:,0].astype(int),j[:,1].astype(int)] *= weight_jxp1jy
    n[j[:,0].astype(int),jp1[:,1].astype(int)] *= weight_jxjyp1
    n[jp1[:,0].astype(int),jp1[:,1].astype(int)] *= weight_jxp1jyp1
    
    #charge density from particles smeared across grid points
    rho = n0*n*Lx*Ly/N/(dx*dy)
    
    #calculate electric potential at each grid point using FFT
    phi_grid = phi_compute(rho,eps0,Nx,Ny,dx,dy)
    
    #calculate electric potential at each grid point using finite difference
    #f = np.zeros((Nx,Ny))
    #phi_grid = phi_solve(f, rho, dx, roll_step, atol=1.E-6, max_steps=10**5)
    
    #calculate electric field from electric potential
    Ex = -np.gradient(phi_grid, dx, axis = 0)
    Ey = -np.gradient(phi_grid, dy, axis = 1)
    E = np.zeros((N,2))
    j = j.astype(int)
    jp1 = jp1.astype(int)
    
    #interpolate electric field at each particle position
    E[:,0] = Ex[j[:,0],j[:,1]]*weight_jxjy + Ex[j[:,0],jp1[:,1]]*weight_jxjyp1 + Ex[jp1[:,0],j[:,1]]*weight_jxp1jy + Ex[jp1[:,0],jp1[:,1]]*weight_jxp1jyp1
    E[:,1] = Ey[j[:,0],j[:,1]]*weight_jxjy + Ey[j[:,0],jp1[:,1]]*weight_jxjyp1 + Ey[jp1[:,0],j[:,1]]*weight_jxp1jy + Ey[jp1[:,0],jp1[:,1]]*weight_jxp1jyp1
    a = -E
    
    return a

    
    
    
    

In [2]:
"""
This cell contains the main function where the main loop of the simulation occurs, updating the position and velocity of
each particle at each time step

"""

def main():
    N = 4*10**4         # number of particles
    Nx = 128            # grid points in x direction
    Ny = 128            # grid points in y direction
    Lx = 10             # grid length in x direction
    Ly = 10             # grid length in y direction
    dx = Lx/Nx          # grid spacing in x direction
    dy = Ly/Ny          # grid spacing in y direction
    t = 0               # initial time
    tEnd = 10           # total time
    dt = 0.1            # time step size
    n0 = 100            # specific weight of each particle
    vb        = 1.5       # beam velocity
    vth       = 1       # beam width
    A         = 0.1     # perturbation when including sinusoidal perturbation
    plotRealTime = True # switch on for plotting as the simulation goes along
        
    
    #initial positions of particles, organized with gaussian noise into two beams
    pos = np.zeros((N,2))
    pos[:np.floor(N/2).astype(int),1] = np.mod(5*Ly/10 + 0.05 * np.random.randn(np.floor(N/2).astype(int)), Ly)
    pos[np.floor(N/2).astype(int):,1] = np.mod(5*Ly/10 + 0.05 * np.random.randn(np.floor(N/2).astype(int)), Ly)
    pos[:,0] = np.random.uniform(0,Lx, size = N)
    
    #initial velocity of each particle, organized with gaussian noise into two beams
    vel = np.zeros((N,2))
    vel[:np.floor(N/2).astype(int),0]  = vth * np.random.randn(np.floor(N/2).astype(int)) + 0.25*vb
    vel[np.floor(N/2).astype(int):,0]  = -vth * np.random.randn(np.floor(N/2).astype(int)) - vb

    #sinusoidal perturbation to initial velocity
    vel[:,0] *= (1 + A*np.sin(2*np.pi*pos[:,0]/Lx))
        
    #calculate inital acceleration
    acc = getAccel(pos,Nx,Ny,Lx,Ly,dt,n0,N)
    
    #calculate new positions and velocities for each time step using leapfrog method
    Nt = int(np.ceil(tEnd/dt))
    for i in tqdm(range(Nt)):
        # (1/2) kick
        vel += acc * dt/2.0
        
        # drift (and apply periodic boundary conditions)
        pos += vel * dt
        pos[:,0] = np.mod(pos[:,0], Lx)
        pos[:,1] = np.mod(pos[:,1], Ly)


        # update accelerations
        acc = getAccel(pos,Nx,Ny,Lx,Ly,dt,n0,N)
        
        # (1/2) kick
        vel += acc * dt/2.0

        # update time
        t += dt
        
        %matplotlib
        # plot in real time - color 1/2 particles blue, other half red
        if plotRealTime or (i == Nt-1):
            plt.cla()
            #plt.scatter(pos[:np.floor(N/2).astype(int),0],pos[:np.floor(N/2).astype(int),1],s=.4,color='blue', alpha=0.5)
            #plt.scatter(pos[np.floor(N/2).astype(int):,0],pos[np.floor(N/2).astype(int):,1],s=.4,color='red', alpha=0.5)
            plt.scatter(pos[:np.floor(N/2).astype(int),0],vel[:np.floor(N/2).astype(int),0],s=.4,color='blue', alpha=0.5)
            plt.scatter(pos[np.floor(N/2).astype(int):,0],vel[np.floor(N/2).astype(int):,0],s=.4,color='red', alpha=0.5)
            plt.axis([0,Lx,-Ly,Ly])
            plt.pause(0.001)
            
            
    # Save figure
    plt.xlabel('$x$')
    plt.ylabel('$v_x$')
    #plt.savefig('two_stream_instability.png',dpi=240)
    plt.show()
    
    return 0



In [3]:
""" 
This cell runs the main program where you will see the plasma simulation
"""
if __name__== "__main__":
  main()


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Using matplotlib backend: <object object at 0x00000163C52EC130>


  2%|█▋                                                                                | 2/100 [00:00<00:22,  4.26it/s]

Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg


  4%|███▎                                                                              | 4/100 [00:00<00:16,  5.80it/s]

...

Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg


 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [00:14<00:00,  7.23it/s]

Using matplotlib backend: QtAgg
Using matplotlib backend: QtAgg


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:14<00:00,  6.95it/s]

Using matplotlib backend: QtAgg
